# Импорты

In [51]:
import asyncio
from openai import AsyncOpenAI
import pandas as pd
import nest_asyncio
import random, json
from tqdm.notebook import tqdm
from more_itertools import batched
from qdrant_client.http.models import (
    Prefetch, FusionQuery, Fusion,
    Filter, FieldCondition, MatchText
)

from openai import OpenAI
import numpy as np
from pathlib import Path
import sys
from qdrant_client import QdrantClient, models, AsyncQdrantClient
sys.path.append(str(Path.cwd().parent))
from app import gen_queries, embed, get_context
from config import LLM_NAME, ENCODER_NAME, COLLECTION_NAME, LLM_PORT, ENCODER_PORT, QDRANT_PORT

nest_asyncio.apply()

# Генерация запросов для тестов

In [ ]:
client = AsyncOpenAI(
    base_url=f"http://localhost:{LLM_PORT}/v1",
    api_key="EMPTY",
    timeout=600
)

async def question(row) -> str:
    resp = await client.chat.completions.create(
        model=LLM_NAME,
        messages=[{"role": "user", "content": row["prompts"]}],
        max_tokens=512,
        temperature=0.1,
        extra_body={
            "chat_template_kwargs": {"enable_thinking": False}
        }
    )
    usage = resp.usage
    
    return {'index': row['index'],
            'question': resp.choices[0].message.content.strip(),
            'completion_tokens': usage.completion_tokens,
            'prompt_tokens': usage.prompt_tokens,
            'total_tokens': usage.total_tokens}

async def question_batch(dct: list) -> list[str]:
    return await asyncio.gather(*(question(row) for row in dct))

async def guestion_in_loop(batches: list):
    results = []
    for batch in tqdm(batches):
        results.extend(await question_batch(batch))
    return results

In [125]:
table_of_articles = pd.read_csv("../data/buffer_table.csv")
table_of_articles['prompts'] = table_of_articles.text.map(lambda x: f'Вот тебе документ: {x.strip()}\n\nТвоя задача составить 3 прямых вопроса на английском по содержанию документа для оценки поисковой системы. Раздели вопросы занком "&&".')
dst_texts = table_of_articles.reset_index()
dst_texts['index'] = dst_texts.index.astype(int)
dst_texts = dst_texts.to_dict(orient='records')
sample = random.sample(dst_texts, 500)

In [126]:
batches = list(batched(sample, 150))
questions = asyncio.run(guestion_in_loop(batches))

  0%|          | 0/4 [00:00<?, ?it/s]

In [136]:
questions = pd.DataFrame(questions)
questions['question'] = questions.question.str.split('&&')
questions = questions.explode('question').drop_duplicates(subset='question')
questions

,index,question,completion_tokens,prompt_tokens,total_tokens
0,4444,How can I set up a Firefox quicksearch to easi...,46,128,174
0,4444,What are some cool tricks you can use in the ...,46,128,174
0,4444,How can I find tracked issues more easily usi...,46,128,174
1,1163,What is the recommended time of day to set an ...,51,279,330
1,1163,"What should be considered when choosing the ""...",51,279,330
...,...,...,...,...,...
498,13089,What is the general setting related to the 's...,57,89,146
498,13089,What is the purpose of the 'admin/setting/the...,57,89,146
499,5260,How do you create a link in a document?,32,75,107
499,5260,What is the purpose of creating a link?,32,75,107


# Кодирование вопросов

In [ ]:
client_qdrant = AsyncQdrantClient(host="localhost", port=QDRANT_PORT, check_compatibility=False, timeout=1000)
Docs_Dense = COLLECTION_NAME


# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = f"http://localhost:{ENCODER_PORT}/v1"

client = OpenAI(
    # defaults to os.environ.get("O|PENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)
#client.models.list()

In [ ]:
embeddings = client.embeddings.create(
    input=questions['question'],
    model=ENCODER_NAME,
)
embeddings = np.array(list(map(lambda x: x.embedding, embeddings.data)))
embeddings.shape

In [141]:
questions

,index,question,completion_tokens,prompt_tokens,total_tokens
0,4444,How can I set up a Firefox quicksearch to easi...,46,128,174
0,4444,What are some cool tricks you can use in the ...,46,128,174
0,4444,How can I find tracked issues more easily usi...,46,128,174
1,1163,What is the recommended time of day to set an ...,51,279,330
1,1163,"What should be considered when choosing the ""...",51,279,330
...,...,...,...,...,...
498,13089,What is the general setting related to the 's...,57,89,146
498,13089,What is the purpose of the 'admin/setting/the...,57,89,146
499,5260,How do you create a link in a document?,32,75,107
499,5260,What is the purpose of creating a link?,32,75,107


# Test запросов

In [53]:
await get_context('hi', 3, 3)

{
  "queries": [
    "Hello!",
    "Greetings!",
    "Hi there!"
  ]
}


(['Hello!', 'Greetings!', 'Hi there!', 'query'],
 QueryResponse(points=[ScoredPoint(id=491, version=0, score=1.3333334, payload={'text': '##Accessibility notes\n\nPHP variables holding \'silent\' (that is, not spoken by assistive technologies including screen readers) representations of right and left arrows (example ► &#x25BA; ), to avoid misuse of characters including "greater than" >, "right angle quote"\xa0»', 'urls': 'https://docs.moodle.org/403/en/Accessibility_notes'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=160, version=0, score=0.8333334, payload={'text': '##APC user cache (APCu)\n\n###About using APCu in Moodle', 'urls': 'https://docs.moodle.org/403/en/APC_user_cache_(APCu)'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=95, version=0, score=0.6666667, payload={'text': '##AMOS manual\n\nAMOS stands for Automated Manipulation Of Strings. AMOS is a central repository of Moodle strings and their history. It tracks the addition of English 

In [60]:
import requests, json
port = 7998

response = requests.post(f"http://localhost:{port}/rag", json={
    "query": "Как создать новый курс в Moodle?",
    "k": 3,
    "top": 5
})

json.loads(response.text)

{'answer': 'Чтобы создать новый курс в Moodle, вам нужно перейти на страницу "Курсы" и нажать кнопку "Добавить новый курс". Затем выберите тип курса (например, "Обучение", "Семинар" или "Тестирование") и заполните необходимые поля. После этого можно добавить модули, задать даты и время занятий, а также настроить доступ для участников.',
 'queries': ['How can I create a new course in Moodle?',
  'What is the process for creating a new course in Moodle?',
  'How do I initiate the creation of a new course within Moodle?',
  'query'],
 'context': {'points': [{'id': 389,
    'version': 0,
    'score': 1.5,
    'payload': {'text': '##Accessibility Toolkit\n\nThis page explains the usage of the Starter Toolkit. If you are looking for information on the enterprise product - please check the Brickfield Education Labs website .',
     'urls': 'https://docs.moodle.org/403/en/Accessibility_Toolkit'},
    'vector': None,
    'shard_key': None,
    'order_value': None},
   {'id': 505,
    'version':

In [ ]:
import requests

# представление
response1 = requests.post(f"http://localhost:{port}/rag", json={
   "query": "Привет, меня зовут Андрей",
   "history": []
})

# проверка памяти
response2 = requests.post(f"http://localhost:{port}/rag", json={
   "query": "Как меня зовут?",
   "history": [
      {"role": "user", "content": "Привет, меня зовут Андрей"},
      {"role": "assistant", "content": response1.json()["answer"]}
   ]
})

# поиск с контекстом
response3 = requests.post(f"http://localhost:{port}/rag", json={
   "query": "Как понять, если у меня права администратора?",
   "history": [
      {"role": "user", "content": "Привет, меня зовут Андрей"},
      {"role": "assistant", "content": response1.json()["answer"]},
      {"role": "user", "content": "Как меня зовут?"},
      {"role": "assistant", "content": response2.json()["answer"]}
   ]
})

print("Первый ответ:", response1.json())
print("Второй ответ:", response2.json())
print("Третий ответ:", response3.json())